In [1]:
import pandas as pd
import os
import numpy as np
import epiweeks
import datetime
import requests
import zipfile
import gzip
import csv
from contextlib import ExitStack
import progressbar as pbar
import eurostat
import json

In [2]:
today = datetime.date.today().strftime("%d-%m-%Y")
data_dir_path = f'data/World/{today}'
if not os.path.exists(f'{data_dir_path}'):
        os.mkdir(f'{data_dir_path}')

In [3]:
read = open('data/Data/old_population.csv', 'r', encoding='utf-8')
reader = csv.reader(read)
countries = {}
for index, row in enumerate(reader):
    if index in [0]:
        continue
    short, country = row[6].split(':')
    if len(short) == 2 and country not in countries.keys():
        if short == 'DE':
            countries['Germany'] = short
        else:
            countries[country] = short
with open(f'{data_dir_path}/eurostat_countries.json', 'w') as file:
    json.dump(countries, file, indent=4)

In [4]:
file_name = 'old_population.csv'
with ExitStack() as stack:
    writer_dict = {}
    for country in countries.keys():
        if not os.path.exists(f'{data_dir_path}/{country}'):
            os.mkdir(f'{data_dir_path}/{country}')
        file_list = stack.enter_context(open(f'{data_dir_path}/{country}/{file_name}', 'w'))
        writer_dict[countries[country]] = csv.writer(file_list, delimiter=',')
        writer_dict[countries[country]].writerow(['Year', 'Region', 'Age group', 'Sex', 'Value'])
    read = stack.enter_context(open('data/Data/old_population.csv', 'r', encoding='utf-8'))
    reader = csv.reader(read)
    for index, row in enumerate(reader):
        if index in [0]:
            continue
        if row[7] >= str(2015) and row[6][:2] in writer_dict.keys():
            writer_dict[row[6][:2]].writerow([row[7], row[6].split(':')[0], row[5].split(':')[0], row[4].split(':')[0], row[8]])
print('Population files prepared')

Population files prepared


In [5]:
file_name = 'old_deaths.csv'
with ExitStack() as stack:
    writer_dict = {}
    for country in countries.keys():
        file_list = stack.enter_context(open(f'{data_dir_path}/{country}/{file_name}', 'w'))
        writer_dict[countries[country]] = csv.writer(file_list, delimiter=',')
        writer_dict[countries[country]].writerow(['Year', 'Region', 'Age group', 'Sex', 'Value'])
    read = stack.enter_context(open('data/Data/old_deaths.csv', 'r', encoding='utf-8'))
    reader = csv.reader(read)
    for index, row in enumerate(reader):
        if index in [0]:
            continue
        if row[7][:4] >= str(2015) and row[6][:2] in writer_dict.keys():
            writer_dict[row[6][:2]].writerow([row[7], row[6].split(':')[0], row[3].split(':')[0], row[4].split(':')[0], row[8]])
print('Deaths files prepared')

Deaths files prepared
